In [2]:
import torch
import pickle, gzip
import sys
import numpy as np
import pandas as pd
import os
from voc import read_object_labels_csv

/home/seongha/anaconda3/envs/MGSSL/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
coco_lt_path = "data/coco"
voc_lt_path = "data/voc/"

# VOC2007

In [4]:
path_csv = './data/voc/files/VOC2007'
trainval_csv = os.path.join(path_csv, 'classification_trainval_LT.csv')
test_csv = os.path.join(path_csv, 'classification_test.csv')

In [6]:
import pandas as pd

In [5]:
hi = pd.read_csv(trainval_csv)
print(hi.columns)
print(hi.shape)



Index(['name', 'aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 'car',
       'cat', 'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike',
       'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor'],
      dtype='object')
(1058, 21)


In [9]:
hi = pd.read_csv(trainval_csv)
print(hi.columns)
print(hi.shape)
## generate gt pkl file
hi=hi.replace(0,1)
hi=hi.replace(-1,0)
gt_labels = hi.iloc[:,1:].to_numpy()
print(gt_labels.shape)
img_id2idx = dict()
idx2img_id = dict()
print(gt_labels)

for i, row in hi.iterrows():
  img_id2idx[row['name']] = i
  idx2img_id[i] = row['name']


Index(['name', 'aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 'car',
       'cat', 'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike',
       'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor'],
      dtype='object')
(1058, 21)
(1058, 20)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 1 0 0]]


In [19]:
res = {
  "gt_labels": gt_labels,
  "img_id2idx": img_id2idx,
  "idx2img_id": idx2img_id
}
with open('test_terse_gt_2007.pkl', 'wb') as f:
  pickle.dump(res, f)

# Generate VOC ADJ PKL file


In [14]:
hi = pd.read_csv(trainval_csv)
print(hi.columns)
print(hi.shape)
## generate gt pkl file
hi=hi.replace(0,1)
hi=hi.replace(-1,0)

hi=hi.iloc[:,1:]
di = dict()
adj = []
nums = []

for col_name, val in hi.iteritems():
  # print(col_name)
  if col_name =='name':
    continue
  if col_name == 'pottedplant':
    M = hi[hi[col_name].isin([1.0])].sum(axis=0)
    print(M)
  # di[col_name] = val.value_counts().to_dict()
  M = hi[hi[col_name].isin([1.0])].sum(axis=0)
  # print(M)
  nums.append(M[col_name])
  M[col_name] = 0
  # print(cond_prob[col_name])
  # print(M)
  adj.append(M.to_list())
  
  # print(hi[hi[col_name].isin([1.0])].sum(axis=0))
# nums.sort()
print(nums)

adj = np.asarray(adj)
nums = np.asarray(nums)

di['adj'] = adj
di['nums'] = nums
print(di['adj'])

# import json
# with open("voc_class_num.json", 'w') as f:
#   json.dump(di, f)


Index(['name', 'aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 'car',
       'cat', 'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike',
       'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor'],
      dtype='object')
(1058, 21)
aeroplane        0
bicycle          0
bird             0
boat             1
bottle           6
bus              0
car              5
cat              6
chair           60
cow              0
diningtable     35
dog              4
horse            0
motorbike        0
person          52
pottedplant    135
sheep            0
sofa            29
train            1
tvmonitor       14
dtype: int64
[4, 30, 6, 12, 58, 13, 247, 27, 428, 8, 207, 40, 18, 17, 627, 135, 3, 133, 6, 99]
[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   4   0   0   0
    0   0]
 [  0   0   0   0   0   1   3   0   4   0   0   0   0   1  27   0   0   1
    0   1]
 [  0   0   0   0   0   0   1   0   1   0   0   0   0   0   4   0   0   0
    0   0]
 [  0   0   

In [ ]:
import pickle
with open('voc_lt_adj.pkl', 'wb') as f:
  pickle.dump(di, f)

In [8]:
##check
# print(nums)
nums2 = nums[:, np.newaxis]
# print(nums2)
# P = adj / nums2
P = np.divide(adj, nums2)
# print("p", P)

t = np.mean(P, axis=0)
np.set_printoptions(3)
# print("t:", t)
P[P < t] = 0 #balance
# print(P)
P[P >= t] = 1
print(P)
P = P * 0.2 / (P.sum(0, keepdims=True) + 1e-6) #reweight
print(P)
P = P + np.identity(len(nums), np.int) * 0.8
print(P)

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 1. 1. 0. 1. 0. 1.]
 [0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0.]
 [0. 1. 1. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 0. 1. 0. 0.]
 [0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 1. 1. 0. 0. 0. 1. 0. 1. 0. 1.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 1. 0. 1. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1.]
 [0. 0. 0. 1. 1. 0. 0. 1. 1. 0. 1. 1. 0. 0. 0. 0. 0. 1.

/tmp/ipykernel_40963/1120479978.py:18: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  P = P + np.identity(len(nums), np.int) * 0.8


In [21]:
with open(voc_lt_path + "/testlongtail2007/img_id.txt", "r") as f:
  # data= pickle.load(f)
  data = f.readlines()
  # print(data)
  data = [int(each) for each in data]
  # data = list(map(str,data))
  print(len(data))
print(max(data))
# print(hi['name'].isin({'name': data}))
# print(hi['name'].dtype)

LT = hi.loc[hi['name'].isin(data)]

LT['name'] = LT['name'].apply(lambda x:format(x, '06d'))
print(LT['name'])
import os
LT_path = os.path.join(path_csv, 'classification_test_' + "LT.csv")
LT.to_csv(LT_path, index=False) 

1366
9963
0       000001
2       000003
3       000004
4       000006
5       000008
         ...  
4941    009941
4944    009951
4948    009957
4950    009962
4951    009963
Name: name, Length: 1366, dtype: object


/tmp/ipykernel_40963/2512746191.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  LT['name'] = LT['name'].apply(lambda x:format(x, '06d'))


# COCO2014

In [13]:
path_csv = './data/coco/'
import json
with open('data/coco/data/category.json') as f:

  category = json.load(f)
  print(category.keys())
  print(len(category.items()))
  print(category.values())

with open('data/coco/data/train_anno.json') as f:

  train = json.load(f)
  print(len(train), train[0])

  li = []
  gt_labels = np.zeros((len(train),80))
  img_id2idx = dict()
  idx2img_id = []
  for i,each in enumerate(train):
    li += each['labels']
    gt_labels[i, each['labels']] = 1
    img_id = int(each['file_name'].split('.')[0].split('_')[-1])
    idx2img_id.append(img_id)
    img_id2idx[img_id] = i
  print(max(li))

  df = pd.DataFrame(gt_labels)
  print(df.columns)
  di = dict()
  for col_name, val in df.iteritems():
    if col_name == 'name': continue
    print(val)
    df[col_name] = val.value_counts().to_dict()
  
  with open('coco_class_num', 'w') as f:
    json.dump(di, f)

with open('data/coco/data/val_anno.json') as f:

  val = json.load(f)
  print(len(val), val[0])

  test_gt_labels = np.zeros((len(val),80))
  
  for i,each in enumerate(val):
    test_gt_labels[i, each['labels']] = 1
  print(test_gt_labels)

res = {
  "gt_labels": gt_labels,
  "img_id2idx": img_id2idx,
  "idx2img_id": idx2img_id,
  "test_gt_labels": test_gt_labels
}

with open('terse_gt_2014.pkl', 'wb') as f:
  pickle.dump(res, f)

dict_keys(['airplane', 'apple', 'backpack', 'banana', 'baseball bat', 'baseball glove', 'bear', 'bed', 'bench', 'bicycle', 'bird', 'boat', 'book', 'bottle', 'bowl', 'broccoli', 'bus', 'cake', 'car', 'carrot', 'cat', 'cell phone', 'chair', 'clock', 'couch', 'cow', 'cup', 'dining table', 'dog', 'donut', 'elephant', 'fire hydrant', 'fork', 'frisbee', 'giraffe', 'hair drier', 'handbag', 'horse', 'hot dog', 'keyboard', 'kite', 'knife', 'laptop', 'microwave', 'motorcycle', 'mouse', 'orange', 'oven', 'parking meter', 'person', 'pizza', 'potted plant', 'refrigerator', 'remote', 'sandwich', 'scissors', 'sheep', 'sink', 'skateboard', 'skis', 'snowboard', 'spoon', 'sports ball', 'stop sign', 'suitcase', 'surfboard', 'teddy bear', 'tennis racket', 'tie', 'toaster', 'toilet', 'toothbrush', 'traffic light', 'train', 'truck', 'tv', 'umbrella', 'vase', 'wine glass', 'zebra'])
80
dict_values([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 3

ValueError: Length of values (2) does not match length of index (82081)

In [13]:
with open('data/coco/data/train_anno_LT.json') as f:
  adj = np.zeros((80,80))
  import json
  train = json.load(f)
  print(len(train), train[0])

  li = []
  gt_labels = np.zeros((len(train),80))
  img_id2idx = dict()
  idx2img_id = []
  for i,each in enumerate(train):
    li += each['labels']
    gt_labels[i, each['labels']] = 1
    img_id = int(each['file_name'].split('.')[0].split('_')[-1])
    idx2img_id.append(img_id)
    img_id2idx[img_id] = i
  nums = gt_labels.sum(axis=0)
  adj = []
  for i,col in enumerate(gt_labels.T):
    cond_prob = gt_labels[np.isin(col,[1.0]),:].sum(axis=0)
    cond_prob[i] = 0
    adj.append(cond_prob)
    print(adj[-1])
  nums.sort()
  print(nums)
  di={'adj': np.asarray(adj), "nums": np.asarray(nums)}


1861 {'file_name': 'COCO_train2014_000000457732.jpg', 'labels': [13, 14, 22, 57, 26, 27]}
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  1.  0.  0.  0.
  2.  0.  0.  0.  2.  0.  0.  0.  1.  1.  0.  0.  0.  0.  0.  0.  0.  0.
  3.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 10.  0.  1.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  3.  0.  0.  0.  0.  0.]
[ 0.  0.  0.  6.  0.  0.  0.  0.  0.  0.  0.  0.  7. 14. 21.  0.  0.  1.
  1.  1.  2.  0. 12.  3.  4.  0. 11. 21.  0.  0.  0.  0.  1.  0.  0.  0.
  0.  0.  0.  0.  0.  5.  1.  5.  0.  0.  7.  6.  0. 10.  1.  4.  7.  1.
  0.  0.  0.  6.  0.  0.  0.  5.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.
  0.  0.  0.  3.  0.  2.  3.  0.]
[ 0.  0.  0.  3.  0.  0.  0.  3.  7.  8.  0.  0. 11. 16.  5.  0.  4.  1.
 18.  1.  1.  2. 27.  1.  8.  1. 16. 20.  5.  0.  0.  1.  2.  2.  1.  0.
 39.  0.  0.  3.  1.  6. 14.  2.  3.  7.  0.  5.  0. 72.  0.  7.  5.  5.
  0.  1.  0.  4.  2.  0.  0.  5.  0.  1

In [50]:
  import pickle
  with open('coco_lt_adj.pkl', 'wb') as f:
    pickle.dump(di, f)
  np.save('coco_class_num.npy', gt_labels.sum(axis=0))

  with open('coco_class_num', 'w') as f:
    json.dump(di, f)

In [64]:
with open("data/coco/img_id.pkl", "rb") as f:
  # data= pickle.load(f)
  data = f.readlines()
  data = [int(each) for each in data]
  # print(data)
  train_anno = []
  with open('data/coco/data/train_anno.json') as f:

    train = json.load(f)
    print(len(train), train[0])
    for each in train:
      img_id = int(each['file_name'].split('.')[0].split('_')[-1])
      if img_id in data:
        train_anno.append(each)
  print(len(train_anno))

  with open("data/coco/data/train_anno_LT.json", "w") as f:
    json.dump(train_anno, f)
  


82081 {'file_name': 'COCO_train2014_000000057870.jpg', 'labels': [12, 77, 51, 22, 27]}
1861
